# calcular_tendencias.py

### Con este programa representamos la frecuencia de cada clase, en una región determinada del hemisferio norte que escojamos, con resspecto al año y obtenemos la regresión lineal de dicha representación, analizando además el p-valor para ver la significancia estadística

#### Cargar modulos

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import pymannkendall as mk

#### Parametros de entrada

In [3]:
DatosEntrada="C:\\Users\\Usuario\\Desktop\\Practicas Meteogalicia\\Datos de entrada\\wtseries_interim_historical_r1i1p1_nh_1979_2005.nc"
lat_galicia=43
lon_galicia=360-9
labelregion="Galcia"
datos_tipos=['PA', 'DANE', 'DAE', 'DASE', 'DAS', 'DASW', 'DAW', 'DANW', 'DAN', 'PDNE', 'PDE', 'PDSE', 'PDS', 'PDSW', 'PDW', 'PDNW', 'PDN', 'PC', 'DCNE', 'DCE', 'DCSE', 'DCS', 'DCSW', 'DCW', 'DCNW', 'DCN', 'U']

In [6]:
datos=xr.open_dataset(DatosEntrada) #El formato es xarray.core.dataset.Dataset
datos_tiempo=datos.time #El formato es xarray.core.dataarray.DataArray
datos_tiempo=datos_tiempo.to_pandas() #El formato es un pandas.core.series.Series
datos_lon=datos.lon.values
datos_lat=datos.lat.values #Estos dos últimos son arrays

datos_galicia = datos.wtseries.sel(lon=lon_galicia, lat=lat_galicia, method='nearest') 
#Este archivo es un xarray.core.dataarray.DataArray



periodo=pd.date_range(start=datos_tiempo[0], end=datos_tiempo[-1], freq="6h")
#El formato es pandas.core.indexes.datetimes.DatetimeIndex

#### En esta parte filtramos la mitad del año de verano

In [5]:
meses=[10,11,12,1,2,3]
labelmeses='ONDEFM'

locmeses=np.where(periodo.month.isin(meses))[0] #El formato es un array

datos_galicia=datos_galicia.isel(time=locmeses)
periodo=periodo[locmeses] #El formato es pandas.core.indexes.datetimes.DatetimeIndex

#### Defino la función de regresión lineal

In [7]:
def func(x, a, b):
    y=a*x+b
    return y


#### Sacamos, para cada clase, la frecuencia con respecto al año y lo guardamos en una matriz

In [8]:
years= np.unique(periodo.year) #El formato es array
freq_año=np.zeros((27, len(years)))
for i in years:
  locaño=np.where(periodo.year.values==i)[0]
  datos_galicia_año=datos_galicia.isel(time=locaño) #El formato es xarray.core.dataarray.DataArray
  freq_año[:, i-1979]=np.histogram(datos_galicia_año.values, bins=np.arange(1,29))[0]
  

#### En esta parte sacamos la frecuencia relativa de cada clase creando una tabla de pandas para la misma, representamos en grafico de barras la frecuencia absoluta con respecto a los años de cada clase, representamos la recta obtenida de una regresión lineal de dichos datos sobre la anterior representación y guardamos cada figura. Finalmente hacemos un Mann Kendall Trend Test para sacar el p-valor de dicha tendencia lineal

In [11]:
freq_relativa=[]
for i in range(0, 27):
    x=years
    y=freq_año[i,:]
    popt, pcov = curve_fit(func, x, y)
    a, b=popt
    MannKendall = mk.original_test(y)
    freq_total=np.sum(y)
    freq_rel=freq_total/len(datos_galicia.time.values)
    freq_relativa.append(freq_rel)
    print('Clase '+str(datos_tipos[i]))
    print('a =',a)
    print('b =',b)
    print('Test MannKendall:', MannKendall)
    figure=plt.figure()
    plt.bar(x, y)
    plt.plot(x, func(x,a,b), color='green')
    plt.xlabel('Año')
    plt.ylabel('Frecuencia')
    plt.title(labelregion+' Clase:'+str(datos_tipos[i]) +'  '+ labelmeses + '  Slope:'+str(np.round(a, decimals=4))+'  p-valor:'+str(np.round(MannKendall[2], decimals=4)))
    nombre_figure=labelregion+'_'+labelmeses+'_'+'_frecuencia_'+str(datos_tipos[i])+'.jpg'
    plt.savefig(nombre_figure)
    plt.close()
    print('=================')
d = {'Clase': datos_tipos, 'Frecuencia relativa': list(freq_relativa)}
tabla_freq_rel=pd.DataFrame(data=d)

Clase PA
a = 0.14163614163426763
b = 80.34228734246369
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.8185845721803675, z=0.22936585546278224, Tau=0.03418803418803419, s=12.0, var_s=2300.0, slope=0.25, intercept=354.75)
Clase DANE
a = 0.41575091574965817
b = -779.5832315849676
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.45256724623198297, z=0.7511419256205195, Tau=0.10541310541310542, s=37.0, var_s=2297.0, slope=0.3, intercept=44.1)
Clase DAE
a = 0.33394383394238114
b = -638.8087098101053
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.4033953048926282, z=0.8355727485873496, Tau=0.1168091168091168, s=41.0, var_s=2291.6666666666665, slope=0.2, intercept=20.4)
Clase DASE
a = 0.18498168497990186
b = -357.1131461139165
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.3447751174036433, z=0.9447723372967624, Tau=0.13105413105413105, s=46.0, var_s=2268.6666666666665, slope=0.09523809523809523, intercept=7.76190476

In [12]:
print(tabla_freq_rel)
datos.close() 
print('script acabo')

   Clase  Frecuencia relativa
0     PA             0.248099
1   DANE             0.033259
2    DAE             0.018074
3   DASE             0.007782
4    DAS             0.005881
5   DASW             0.025857
6    DAW             0.035921
7   DANW             0.029203
8    DAN             0.026440
9   PDNE             0.063096
10   PDE             0.045097
11  PDSE             0.018150
12   PDS             0.016984
13  PDSW             0.058406
14   PDW             0.069104
15  PDNW             0.051638
16   PDN             0.047353
17    PC             0.065732
18  DCNE             0.009202
19   DCE             0.008188
20  DCSE             0.005628
21   DCS             0.006439
22  DCSW             0.011838
23   DCW             0.009760
24  DCNW             0.006870
25   DCN             0.008416
26     U             0.067583
script acabo
